# Тестване

Осигуряването и управлението на качеството е важен аспект от разработката на софтуер. За целта се използват различни методи, които главно се делят на end-to-end, integration и unit тестове. За първите две отговорността обикновено е у QA инженера, докато unit тестовете трябва да се пишат от разработчиците, понеже са white-box тестове (т.е. тестващия трябва да има знание всеки компонент вътрешно как изглежда).

![различни видове тестове](assets/tests.png)

Целта на unit test-овете е да проиграят всички възможни ключови ситуации, свързани с **един** конкретен компонент и да проверят дали се държи очаквано при тях.

## Before

В папката `13 - Modules` има пример за тестване на един компонент, макар и примитвен такъв. Намира се в `game.engine` модула:

```python
if __name__ == "__main__":
    # Executed when running `python3 -m game.engine`
    
    from game.players.mock_player import MockPlayer

    print("Testing win case...")
    player_win = MockPlayer(1, "oba")
    engine_win = BesenitsaEngine("foobar", player_win)

    assert engine_win.guess() == GameState.ONGOING
    assert engine_win.guess() == GameState.ONGOING
    assert engine_win.guess() == GameState.WON
    print("Test OK.")

    print("Testing lose case...")
    player_lose = MockPlayer(3, "asdg")
    engine_lose = BesenitsaEngine("foobar", player_lose)

    assert engine_lose.guess() == GameState.ONGOING
    assert engine_lose.guess() == GameState.ONGOING
    assert engine_lose.guess() == GameState.ONGOING
    assert engine_lose.guess() == GameState.LOST
    print("Test OK.")
```

Тестването по този начин, освен че може да бъде по-трудоемко и по-трудно за проследяване и изпълнение за по-сложни ситуации и проекти, има и друг недостатък, че когато нещо не е както трябва (някой `assert` не минава), нямаме детайлна информация за това кое не е правилно. Например, ако `engine_win.guess()` връща `GameState.LOST` вместо `GameState.WON`, ще получим само `AssertionError`, без да знаем какъв е точно върнатия резултат на `engine_win.guess()` и защо не е равен на `GameState.WON`.

Още повече, най-главния недостатък на този начин на "тестване" е че така се изпълняват последователно тестовете и ако един гръмне, то другите няма да се изпълнят. А ние не искаме това да е така - трябва всеки тест да се изпълнява независимо от другите - за предпочитане в отделни нишки, без споделена памет и без споделено състояние и т.н.

## After (a.k. unit testing frameworks in Python)

Съществуват няколко основни test runner-а в Python света, като `unittest`, `pytest`, `nose`, `nose2` и други. Ще разгледаме двата най-използвани - `unittest` и `pytest`.

### `unittest`

`unittest` е ***вградена*** библиотека (от Python 2.1 насам), която ни предоставя както framework, така и runner за тестове.

### Особености

* Всеки тестови случай е метод на клас, наследяващ `unittest.TestCase`
* Използват се `assert...` методи на класа вместо `assert` ключовата дума
* Трябва всеки тестови модул да извика `unittest.main()` когато бъде изпълнен директно

Ако трябва горните два test case-a да ги пренапишем с `unittest`, ще изглеждат по този начин:

```python
import unittest

from game.players.mock_player import MockPlayer
from game.engine import BesenitsaEngine, GameState

class EngineTests(unittest.TestCase):
    def test_foobar_win(self):
        player_win = MockPlayer(1, "oba")
        engine_win = BesenitsaEngine("foobar", player_win)

        self.assertEqual(engine_win.guess(), GameState.ONGOING)
        self.assertEqual(engine_win.guess(), GameState.ONGOING)
        self.assertEqual(engine_win.guess(), GameState.WON)

    def test_foobar_lose(self):
        player_lose = MockPlayer(3, "asdg")
        engine_lose = BesenitsaEngine("foobar", player_lose)

        self.assertEqual(engine_lose.guess(), GameState.ONGOING)
        self.assertEqual(engine_lose.guess(), GameState.ONGOING)
        self.assertEqual(engine_lose.guess(), GameState.ONGOING)
        self.assertEqual(engine_lose.guess(), GameState.LOST)

if __name__ == '__main__':
    unittest.main()
```


`self.assertEqual` е един от многото методи, които предоставя `unittest` за проверка на различни условия. Пълен списък може да намерите [тук](https://docs.python.org/3/library/unittest.html#assert-methods). Те са:

* `assertEqual(a, b)` - проверява дали `a == b`
* `assertTrue(x)` - проверява дали `bool(x) is True`
* `assertFalse(x)` - проверява дали `bool(x) is False`
* `assertIs(a, b)` - проверява дали `a is b`
* `assertIsNot(a, b)` - проверява дали `a is not b`
* `assertIsNone(x)` - проверява дали `x is None`
* `assertIsNotNone(x)` - проверява дали `x is not None`
* `assertIn(a, b)` - проверява дали `a in b`
* `assertNotIn(a, b)` - проверява дали `a not in b`
* `assertIsInstance(a, b)` - проверява дали `isinstance(a, b)`
* `assertNotIsInstance(a, b)` - проверява дали `not isinstance(a, b)`

Ползата им можем да видим например ако променим във `test_foobar_win` да очакваме накрая `GameState.LOST` вместо `GameState.WON` (или по някакъв друг начин променим кода така, че да имаме грешно поведение спрямо тестовете). Това output-ът от този тест ще е достатъчно информативен (сравнението показва всички разлики между двата обекта):

```
======================================================================
FAIL: test_foobar_win (test_engine.EngineTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/alexander.ignatov/Desktop/besenitsa/tests/test_engine.py", line 25, in test_foobar_win
    self.assertEqual(engine_win.guess(), GameState.LOST)
AssertionError: <GameState.WON: 1> != <GameState.LOST: 2>
```

### Команди

Директно изпълнение на един файл с тестове:

In [12]:
!python3 test_single_simple_unittest.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Изпълнение на един модул с тестове:

*Важно*: ⚠️ При достъп през пакет (т.е. с точка, както по-долу имаме пакетът `tests`, в който се намира `test_engine` модула) трябва пакетът да съдържа `__init__.py` файл, макар и празен (това показва, че пакетът **не е** само т.нар. **namespace** package).

In [13]:
!python3 -m unittest tests.test_engine

.....
----------------------------------------------------------------------
Ran 5 tests in 0.000s

OK


Автоматично откриване на всички тестове в *текущата* директория:

(търси всички python файлове, започващи с `test`, и ги изпълнява)

In [14]:
!python3 -m unittest discover

.......
----------------------------------------------------------------------
Ran 7 tests in 0.000s

OK


Автоматично откриване на всички тестове в *дадена* директория (в случая `tests`):

In [15]:
!python3 -m unittest discover -s tests

......
----------------------------------------------------------------------
Ran 6 tests in 0.000s

OK


Изпълнение на всички файлове с име, започващо с  `test_`, в директория, наречена `tests`:

In [16]:
!python3 -m unittest discover -s tests -p "test_*.py"

......
----------------------------------------------------------------------
Ran 6 tests in 0.000s

OK


Вербозен output се дава с добавяне на параметъра `-v`:

In [17]:
!python3 -m unittest discover -s tests -v

test_firstGuess_isE (test_ai_player.TestAIPlayer) ... ok
test_cat_lose (test_engine.EngineTests) ... ok
test_cat_win (test_engine.EngineTests) ... ok
test_foobar_lose (test_engine.EngineTests) ... ok
test_foobar_win (test_engine.EngineTests) ... ok
test_initialWord_isMaskedCorrectly (test_engine.EngineTests) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.000s

OK


Ако кодът не се намира в корена на директорията, а например в папка, наречена `src`, можем да добавим параметър `-t src`, за да кажем на `unittest` да изпълни тестовете оттам:

```bash
python3 -m unittest discover -s tests -t src
```

### `pytest`